In [1]:
!git clone https://github.com/sumanthd17/image-descriptor.git

Cloning into 'image-descriptor'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 156 (delta 76), reused 136 (delta 56), pack-reused 0
Receiving objects: 100% (156/156), 158.22 KiB | 3.23 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [2]:
!wget http://images.cocodataset.org/zips/train2014.zip
!unzip train2014

Streaming output truncated to the last 5000 lines.
 extracting: train2014/COCO_train2014_000000013714.jpg  
 extracting: train2014/COCO_train2014_000000194043.jpg  
 extracting: train2014/COCO_train2014_000000219859.jpg  
 extracting: train2014/COCO_train2014_000000278135.jpg  
 extracting: train2014/COCO_train2014_000000141015.jpg  
 extracting: train2014/COCO_train2014_000000280923.jpg  
 extracting: train2014/COCO_train2014_000000200024.jpg  
 extracting: train2014/COCO_train2014_000000435713.jpg  
 extracting: train2014/COCO_train2014_000000249993.jpg  
 extracting: train2014/COCO_train2014_000000424160.jpg  
 extracting: train2014/COCO_train2014_000000142761.jpg  
 extracting: train2014/COCO_train2014_000000532668.jpg  
 extracting: train2014/COCO_train2014_000000564904.jpg  
 extracting: train2014/COCO_train2014_000000346384.jpg  
 extracting: train2014/COCO_train2014_000000560934.jpg  
 extracting: train2014/COCO_train2014_000000122356.jpg  
 extracting: train2014/COCO_train2014

In [3]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!unzip annotations_trainval2014

--2020-07-07 09:51:27--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.109.155
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.109.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M  87.2MB/s    in 2.8s    

2020-07-07 09:51:30 (87.2 MB/s) - ‘annotations_trainval2014.zip’ saved [252872794/252872794]

Archive:  annotations_trainval2014.zip
  inflating: annotations/instances_train2014.json  
  inflating: annotations/instances_val2014.json  
  inflating: annotations/person_keypoints_train2014.json  
  inflating: annotations/person_keypoints_val2014.json  
  inflating: annotations/captions_train2014.json  
  inflating: annotations/captions_val2014.json  


In [4]:
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4139  100  4139    0     0  21899      0 --:--:-- --:--:-- --:--:-- 21899
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
     |████████████████████████████████| 61kB 2.7MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                             

In [3]:
# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

In [4]:
dev = xm.xla_device()

In [5]:
dev

device(type='xla', index=1)

In [6]:
cd image-descriptor

/content/image-descriptor


In [7]:
from utils.vocabulary import Vocabulary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
v = Vocabulary(5, vocab_file='vocab.pkl', from_vocab_file=False, annotation_file='data/annotations/captions_train2014.json')

loading annotations into memory...
Done (t=1.16s)
creating index...


548it [00:00, 5475.75it/s]

index created!


414113it [00:55, 7414.10it/s]


In [8]:
import math
import time
import sys
import os

import torch
import torch.nn as nn
import torch.utils.data as data

import numpy as np

from utils.transforms import transform_train, transform_val
from utils.dataloader import dataloader

from models.encoders import ResNet50
from models.decoders import TextualHead

In [9]:
PRINT_EVERY = 1000

In [10]:
def train(
    train_loader,
    encoder,
    decoder,
    criterion,
    optimizer,
    vocab_size,
    epoch,
    total_step,
    start_step=1,
    start_loss=0.0,
):
    encoder.train()
    decoder.train()

    total_loss = start_loss

    start_train_time = time.time()

    for i_step in range(start_step, total_step + 1):
        indices = train_loader.dataset.get_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        train_loader.batch_sampler.sampler = new_sampler

        for batch in train_loader:
            images, captions = batch[0], batch[1]
            break
        mask = captions.shape[1] * torch.ones(32)

        # move to TPU
        images = images.to(dev)
        captions = captions.to(dev)
        mask = mask.to(dev)

        features = encoder(images)
        outputs = decoder(features, captions, mask)

        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        stats = "Epoch %d, Train step [%d/%d], %ds, Loss: %.4f, Perplexity: %5.4f" % (
            epoch,
            i_step,
            total_step,
            time.time() - start_train_time,
            loss.item(),
            np.exp(loss.item()),
        )
        print("\r" + stats, end="")
        sys.stdout.flush()

        if i_step % PRINT_EVERY == 0:
            print("\r" + stats)
            filename = os.path.join(
                "./models", "train-model-{}{}.pkl".format(epoch, i_step)
            )
            torch.save(
                {
                    "encoder": encoder.state_dict(),
                    "decoder": decoder.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "total_loss": total_loss,
                    "epoch": epoch,
                    "train_step": i_step,
                },
                filename,
            )
            start_train_time = time.time()

    return total_loss / total_step

In [12]:
dl = dataloader(
    mode="train",
    transform=transform_val,
    batch_size=128,
    vocab_threshold=5,
    vocab_file="./pickle/vocab.pkl",
    from_vocab_file=True,
    img_dir_path="./data/train2014",
    captions_path="./data/annotations/captions_train2014.json",
)

loading annotations into memory...
Done (t=0.91s)
creating index...



  0%|          | 672/414113 [00:00<01:01, 6710.25it/s]

index created!



100%|██████████| 414113/414113 [00:54<00:00, 7549.47it/s]


In [13]:
visual = ResNet50(1024)
textual = TextualHead(len(dl.dataset.vocab), 1024, 1, 16, 4096, 0.1, 50, 0)

In [14]:
criterion = nn.CrossEntropyLoss()

params = list(visual.parameters()) + list(textual.parameters())
optimizer = torch.optim.Adam(params=params, lr=0.01)

In [15]:
# move to TPU
visual.to(dev)
textual.to(dev)
criterion.to(dev)

CrossEntropyLoss()

In [16]:
total_val_step = math.ceil(len(dl.dataset.caption_lengths) / 128)

In [ ]:
train(
    dl, visual, textual, criterion, optimizer, len(dl.dataset.vocab), 1, total_val_step
)

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

